In [2]:
import pandas as pd
import numpy as np

In [7]:
# train = pd.read_csv("/content/drive/MyDrive/colab/hackfinal/train_part_small_8x.csv")
train = pd.read_csv("train.csv")

In [ ]:
train = train.fillna("None")
seg_stat = train[["oblast", "Segment"]].groupby("oblast").count()["Segment"]
areas = pd.DataFrame(dict(area=seg_stat.keys(), total=seg_stat.values))
# pd.merge(areas, train[train["Segment"] == 1][["oblast", "Segment"]].groupby("oblast").count()["Segment"].values)
for i in range(5):
    seg_stat = train[train["Segment"] == i + 1][["oblast", "Segment"]].groupby("oblast").count()["Segment"]
    areas = pd.merge(areas, pd.DataFrame({"area": seg_stat.keys(), "segment_{}".format(i+1): seg_stat.values}), on="area", how='left', suffixes=None)

train["os_version"] = train["osv"].apply(lambda x: str(x).split(".")[0])
os_stat = train[["os", "os_version", "Segment"]].groupby(["os", "os_version"]).count()
os_stat = pd.DataFrame(dict(os=list(os_stat["Segment"].keys()), total=os_stat["Segment"].values))
for i in range(5):
    os_seg_stat = train[train["Segment"] == i+1][["os", "os_version", "Segment"]].groupby(["os", "os_version"]).count()
    os_seg_stat = pd.DataFrame({"os": list(os_seg_stat["Segment"].keys()), "segment_{}".format(i+1): os_seg_stat["Segment"].values})
    os_stat = pd.merge(os_stat, os_seg_stat, on="os", how='left', suffixes=None)

areas_stat = areas.copy()
r = list(areas_stat[areas_stat["total"] < 1000].sum())
r[0] = "Прочее"
areas_stat = areas_stat[areas_stat["total"] >= 1000]
areas_stat.loc[len(areas_stat)] = r


r = list(os_stat[os_stat["total"] < 1000].sum())
r[0] = "Прочее"
os_stat.loc[len(os_stat)] = r
os_stat = os_stat[os_stat["total"] >= 1000]

areas_stat.to_csv("area.csv", index=False)
os_stat.to_csv("os_stat.csv", index=False)